# DB2-Salesforce connector: Basic tool information updates

In [1]:
# Parameters
api_url = '/services/data/v43.0/sobjects'
external_id = 'Tool_name__c'
object_id = 'nanoHUB_tools__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",  
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain tool information from DB2 

In [10]:
# Basic tool information
sql_query = "select toolname, title, published, registered from jos_tool"

# display
print(sql_query)

select toolname, title, published, registered from jos_tool


In [11]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine, parse_dates=['registered'])

# display
df.head(2)

,toolname,title,published,registered
0,ellipsom,title-ellipsom,0,2005-03-05 03:02:47
1,hydrolab,Hydrophobicity Lab,1,2006-02-24 03:38:23


In [37]:
df = df.sort_values(by='registered')
display(df.head(5))
display(df.tail(50))
print(df.shape)

,toolname,title,published,registered
348,matlab,Matlab,0,2000-10-02 02:34:37
350,octave,Octave,0,2001-04-04 19:03:01
347,gamess,GAMESS,0,2003-01-15 15:41:22
352,rebo,REBO-1,0,2003-05-08 00:41:33
0,ellipsom,title-ellipsom,0,2005-03-05 03:02:47


,toolname,title,published,registered
1665,youngsmod,Linear Regression Young's modulus,1,2020-09-17 02:17:45
1666,haynes2020,Haynes Machine Learning Tools 2020,0,2020-09-23 16:27:02
1667,amidespec,Amide I Vibrational Spectroscopy Lab,1,2020-09-24 01:34:09
1668,matlabrad,Matlab Coding in the Context of Radiation Hard...,1,2020-09-29 20:39:49
1669,nhremote,Introduction to nanohub remote,1,2020-10-01 19:27:51
1670,thermalcond,Thermal Conductivity Simulator,0,2020-10-04 01:45:16
1671,dftexplorer,DFT Results Explorer,0,2020-10-04 16:20:57
1672,mdtensiletest,Nanoscale tensile tests with MD,0,2020-10-05 18:38:16
1673,ex2pnjunction,PNJunction Lab Exploration Tool,1,2020-10-06 19:22:33
1674,fuelcell,Fuel Cells (Hydrogen-Based Energy Conversion),0,2020-10-13 04:02:01


(1677, 4)


In [12]:
# debugging
print(df['toolname'].to_list().index('citrinednn'))
print(df.iloc[df['toolname'].to_list().index('citrinednn'),:])

1617
toolname                                citrinednn
title         Hands-on Deep Learning for Materials
published                                        1
registered                     2020-05-27 14:49:06
Name: 1617, dtype: object


In [20]:
'mldefect' in df['toolname'].to_list()

False

In [13]:
# Basic tool information
sql_query = "select * from jos_tool_version_alias"

df_jtva = pd.read_sql_query(sql_query, engine)

# display
df_jtva.head(2)

,tool_version_id,alias
0,977,2ds
1,978,abinit


In [21]:
'mldefect' in df_jtva['alias'].to_list()

False

## Building new SQL queries that roll up instances

In [22]:
# pull all tools in DB2
sql_tools = 'select toolname, instance from nanohub.jos_tool_version;'
tools_df = pd.read_sql_query(sql_tools,engine)
display(tools_df.head(5))
print(tools_df.shape)

,toolname,instance
0,112016a,112016a_dev
1,1dbtetransient,1dbtetransient_dev
2,1dchainmd,1dchainmd_dev
3,1dchainmd,1dchainmd_r13
4,1dchainmd,1dchainmd_r15


(4796, 2)


In [23]:
'mldefect' in tools_df['toolname'].to_list()

False

In [29]:
## debugging for Tanya
# jos_resource_stats_clusters
ql = 'select toolname from jos_resource_stats_clusters;'
df_ql = pd.read_sql_query(ql,engine)
'mldefect' in df_ql['toolname'].to_list()


False

In [30]:
df_ql.head(5)

,toolname
0,1dchainmd
1,1dchainmd
2,1dchainmd
3,1dchainmd
4,1dchainmd


In [31]:
# jos_tool_authors
ql = 'select toolname from jos_tool_authors;'
df_ql = pd.read_sql_query(ql,engine)
'mldefect' in df_ql['toolname'].to_list()



False

In [32]:
df_ql.head(5)

,toolname
0,1dchainmd
1,1dchainmd
2,1dchainmd
3,1dchainmd
4,1dfdmht


In [33]:
## nanohub_metrics trials
# metrics_tool_version
ql_metrics = 'select toolname from nanohub_metrics.metrics_tool_version;'
mdf = pd.read_sql_query(ql_metrics,engine)

'mldefect' in mdf['toolname'].to_list()

False

In [39]:
display(mdf.head(5))
display(mdf.tail(5))
print(mdf.shape)

,toolname
0,1dchainmd
1,1dchainmd
2,1dfdmht
3,1dfs
4,1dhetero


,toolname
1867,xfig
1868,xhub
1869,xterm
1870,xterm
1871,yambo


(1872, 1)


In [45]:
mdf['toolname'].to_list()[730:750]

['mimi',
 'minimol',
 'minimol',
 'minimol',
 'minimol',
 'minimol',
 'minimos',
 'minimos',
 'minimos',
 'mmfl',
 'mmst',
 'mmst',
 'mmst',
 'mmst',
 'mmst',
 'mmsttf',
 'mmsttf',
 'mmsttf',
 'mmsttf',
 'moca']

In [47]:
# create a dict strictly of toolnames and their instances
grouped_tools = tools_df.groupby(['toolname']).groups #this is a dict already; need to change the value pairs
import numpy as np
tn_instances = tools_df['instance'].to_numpy()
for i,j in grouped_tools.items():
    grouped_tools[i] = tuple(tn_instances[j])

In [48]:
print(grouped_tools)

{'112016a': ('112016a_dev',), '1dbtetransient': ('1dbtetransient_dev',), '1dchainmd': ('1dchainmd_dev', '1dchainmd_r13', '1dchainmd_r15'), '1dfdmht': ('1dfdmht_dev', '1dfdmht_r24', '1dfdmht_r26', '1dfdmht_r28'), '1dfs': ('1dfs_dev',), '1dhetero': ('1dhetero_dev', '1dhetero_r10', '1dhetero_r13', '1dhetero_r14', '1dhetero_r298', '1dhetero_r299', '1dhetero_r30', '1dhetero_r301', '1dhetero_r303', '1dhetero_r309', '1dhetero_r531', '1dhetero_r532', '1dhetero_r540', '1dhetero_r600', '1dhetero_r689', '1dhetero_r694', '1dhetero_r733', '1dhetero_r739', '1dhetero_r740', '1dhetero_r742'), '1dmd': ('1dmd_dev',), '1dnegf': ('1dnegf_dev',), '1doscillationla': ('1doscillationla_dev',), '1dphononbte': ('1dphononbte_dev', '1dphononbte_r5', '1dphononbte_r9'), '1sl': ('1sl_dev',), '2degpot': ('2degpot_dev',), '2dmatstacks': ('2dmatstacks_dev', '2dmatstacks_r33', '2dmatstacks_r36'), '2dreflect': ('2dreflect_dev', '2dreflect_r29', '2dreflect_r44'), '2ds': ('2ds', '2ds_dev'), '2wth': ('2wth_dev',), '3datomsm

In [53]:
print(list(grouped_tools.keys())[2])
print(grouped_tools[list(grouped_tools.keys())[2]])

1dchainmd
('1dchainmd_dev', '1dchainmd_r13', '1dchainmd_r15')


In [62]:
## Run a separate query for each grouped tool
for key in list(grouped_tools.keys())[1:2]:
    print(key)
    print(grouped_tools[key])
    t_sql_tspec = "select count(user) from nanohub_metrics.toolstart where tool in " +"('1dphononbte_dev', '1dphononbte_r5', '1dphononbte_r9')"+";" #temp sql tool specific
    t_r_df = pd.read_sql_query(t_sql_tspec,engine)
    display(t_r_df.head(5))

1dbtetransient
('1dbtetransient_dev',)


,user
0,jsudibyo
1,jsudibyo
2,jsudibyo
3,jsudibyo
4,jsudibyo


In [63]:
t_r_df.shape

(1714, 1)

In [12]:
# Lifetime usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat')\
group by v.toolname"

toolstart_lifetime_df = pd.read_sql_query(sql_query, engine)

# last year usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=365)).strftime('%Y-%m-%d')

toolstart_last_year_df = pd.read_sql_query(sql_query, engine)

# last 3-month usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=90)).strftime('%Y-%m-%d')

toolstart_last_3_month_df = pd.read_sql_query(sql_query, engine)

# last month usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=30)).strftime('%Y-%m-%d')

toolstart_last_month_df = pd.read_sql_query(sql_query, engine)

# last week usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=7)).strftime('%Y-%m-%d')

toolstart_last_week_df = pd.read_sql_query(sql_query, engine)

# last 24 hours usage count
sql_query = "select v.toolname, count(*) as total, count(distinct t.user) as user_total from nanohub_metrics.toolstart as t \
inner join nanohub.jos_tool_version as v \
on v.instance = t.tool \
where t.user not in ('instanton', 'gridstat') and t.datetime >= '%s'\
group by v.toolname" % (datetime.datetime.now()-datetime.timedelta(days=1)).strftime('%Y-%m-%d')

toolstart_last_24_hours_df = pd.read_sql_query(sql_query, engine)

KeyboardInterrupt: 

In [ ]:
# display
toolstart_last_year_df.head(2)

In [8]:
# insert user and run stats into df
all_df = df.copy()

all_df = pd.merge(all_df, toolstart_lifetime_df.rename(columns={\
                                                                'total':        'Total_runs_over_lifetime',
                                                                'user_total':'Total_users_over_lifetime'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_year_df.rename(columns={\
                                                                'total':        'Total_runs_last_year',
                                                                'user_total':'Total_users_last_year'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_3_month_df.rename(columns={\
                                                                'total':        'Total_runs_last_3_month',
                                                                'user_total':'Total_users_last_3_month'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_month_df.rename(columns={\
                                                                'total':        'Total_runs_last_month',
                                                                'user_total':'Total_users_last_month'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_week_df.rename(columns={\
                                                                'total':        'Total_runs_last_week',
                                                                'user_total':'Total_users_last_week'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')
all_df = pd.merge(all_df, toolstart_last_24_hours_df.rename(columns={\
                                                                'total':        'Total_runs_last_24_hours',
                                                                'user_total':'Total_users_last_24_hours'}\
                                                      ), how='left', left_on='toolname', right_on='toolname')

# display
all_df.head(5).T

,0,1,2,3,4
toolname,ellipsom,hydrolab,huckel,nanomos,sete
title,title-ellipsom,Hydrophobicity Lab,Huckel-IV,NanoMOS,SETE
published,0,1,1,1,0
registered,2005-03-05 03:02:47,2006-02-24 03:38:23,2006-03-01 18:02:30,2006-03-01 18:16:18,2006-03-01 18:16:59
Total_runs_over_lifetime,NaN,3311,NaN,98847,141
Total_users_over_lifetime,NaN,400,NaN,3489,26
Total_runs_last_year,NaN,7,NaN,15141,NaN
Total_users_last_year,NaN,4,NaN,158,NaN
Total_runs_last_3_month,NaN,1,NaN,8978,NaN
Total_users_last_3_month,NaN,1,NaN,79,NaN


## Match data with Salesforce format

In [9]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Name']              = all_df['title']
df_sf['Tool_name__c']  = all_df['toolname']
df_sf['Tool_URL__c']     = all_df['toolname'].apply(lambda x: 'https://nanohub.org/tools/'+x)
df_sf['Published__c']    = all_df['published']

# stats
df_sf['Total_runs_over_lifetime__c']  = all_df['Total_runs_over_lifetime']
df_sf['Total_runs_last_year__c']        = all_df['Total_runs_last_year']
df_sf['Total_runs_last_3_month__c'] = all_df['Total_runs_last_3_month']
df_sf['Total_runs_last_month__c']     = all_df['Total_runs_last_month']
df_sf['Total_runs_last_week__c']       = all_df['Total_runs_last_week']
df_sf['Total_runs_last_24_hours__c'] = all_df['Total_runs_last_24_hours']

df_sf['Total_users_over_lifetime__c']  = all_df['Total_users_over_lifetime']
df_sf['Total_users_last_year__c']        = all_df['Total_users_last_year']
df_sf['Total_users_last_3_month__c'] = all_df['Total_users_last_3_month']
df_sf['Total_users_last_month__c']     = all_df['Total_users_last_month']
df_sf['Total_users_last_week__c']       = all_df['Total_users_last_week']
df_sf['Total_users_last_24_hours__c'] = all_df['Total_users_last_24_hours']

# solidify time-related columns from datetime to string
df_sf['Creation_date__c']  = all_df['registered'].dt.date.fillna('').astype('str')

df_sf.fillna(0, inplace=True)

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,title-ellipsom,Hydrophobicity Lab
Tool_name__c,ellipsom,hydrolab
Tool_URL__c,https://nanohub.org/tools/ellipsom,https://nanohub.org/tools/hydrolab
Published__c,0,1
Total_runs_over_lifetime__c,0,3311
Total_runs_last_year__c,0,7
Total_runs_last_3_month__c,0,1
Total_runs_last_month__c,0,1
Total_runs_last_week__c,0,0
Total_runs_last_24_hours__c,0,0


## To Salesforce Sales Cloud CRM

In [10]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [11]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M0kO7AAJ
[Success] CSV upload successful. Job ID = 7505w00000M0kO7AAJ
[Success] Closing job successful. Job ID = 7505w00000M0kO7AAJ


In [12]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000M0kO7AAJ',
 'operation': 'upsert',
 'object': 'nanoHUB_tools__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-11T08:52:20.000+0000',
 'systemModstamp': '2020-05-11T08:52:21.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Tool_name__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}